In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations
import copy

import bo_methods_lib
# from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle

from pympler import asizeof
import gpflow
import tensorflow_probability as tfp

from matplotlib import pyplot as plt

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-12 17:13:20.093388: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-12 17:13:20.093456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-12 17:13:20.095248: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-12 17:13:20.104615: I tensorflow/core/platform/cpu_feature_guard

In [ ]:
#Set Parameters
cs_name_val = 17
ep0 = 1 #Set initial ep as an even mix between exploration and exploitation
ep_enum_val = 1
meth_name_val = 6
sep_fact = 1.0
gen_heat_map_data = True
normalize = True
noise_mean = 0
noise_std = None
kernel_enum_val = 1
lenscl = None #list([0.136113749, 221.573761, 830.968019, 1.67917241, 0.3, 0.2])
outputscl = None
retrain_GP = 1
reoptimize_obj = 1
bo_iter_tot = 2
bo_run_total = 2
runs_per_job_max = 5
bo_runs_in_job = 2
save_data = False
ei_tol = 1e-7
obj_tol = 1e-7
num_x_data = 10
gen_meth_theta = 1 
gen_meth_x = 2
gen_meth_theta_val = 2
num_val_pts = 20
num_theta_multiplier = 2 #How many simulation data points to generate is equal to num_theta_multiplier*number of parameters
seed = 1

In [ ]:
# Define method, ep_enum classes, indecies to consider, and kernel
meth_name = Method_name_enum(meth_name_val)
method = GPBO_Methods(meth_name)
ep_enum = Ep_enum(ep_enum_val)
kernel = Kernel_enum(kernel_enum_val)
lenscl = lenscl
try:
    lenscl = json.loads(lenscl)
except:
    lenscl = lenscl

# Define Simulator Class (Export your Simulator Object Here)
# simulator = simulator_helper_test_fxns(
#     cs_name_val, noise_mean, noise_std, seed
# )
#All simulator objects will have the same seed. This keeps restarts/jobs consistent for data generation
simulator = simulator_helper_test_fxns(
    cs_name_val, noise_mean, noise_std, 1
)

# Generate Exp Data (OR Add your own experimental data as a Data class object)
set_seed = 1  # Set set_seed to 1 for data generation
gen_meth_x = Gen_meth_enum(gen_meth_x)
if cs_name_val == 16:
    x_vals = np.array([0.0,0.1115,0.2475,0.4076,0.5939,0.8230,0.9214,0.9296,0.985,1.000])
elif cs_name_val == 17:
    x_vals = np.array([0.0087,0.0269,0.0568,0.1556,0.2749,0.4449,0.661,0.8096,0.9309,0.9578])    
else:
    x_vals = None

if cs_name_val in [16,17]:
    noise_std = 0.01
else:
    noise_std = 0.05

exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x, set_seed, x_vals, noise_std)

#Check to make sure x_vals and y_vals are correct
# print(exp_data.x_vals)
# print(exp_data.y_vals)

# Set simulator noise_std artifically as 5% of y_exp mean (So that noise will be set rather than trained)
simulator.noise_std = np.abs(np.mean(exp_data.y_vals)) * noise_std

# Create Exploration Bias Class
if ep_enum.value == 1:
    # Constant value stays constant
    ep_bias = Exploration_Bias(
        ep0, None, ep_enum, None, None, None, None, None, None, None
    )
elif ep_enum.value == 2:
    # For decay method, decay from mixed to full exploitation (alpha of 0.5) for this example
    ep_bias = Exploration_Bias(
        ep0,
        None,
        ep_enum,
        None,
        bo_iter_tot,
        None,
        0.5,
        None,
        None,
        None,
    )
elif ep_enum.value == 3:
    # Set ep multiplier to 1.5 as recommended in Boyle
    ep_bias = Exploration_Bias(
        ep0, None, ep_enum, None, None, 1.5, None, None, None, None
    )
elif ep_enum.value == 4:
    # Jasrasaria method will take care of itself
    ep_bias = Exploration_Bias(
        None, None, ep_enum, None, None, None, None, None, None, None
    )
else:
    raise Warning("Ep_enum value must be between 1 and 4!")

# Generate Sim (Training) Data (OR Add your own training data here as a Data class object)
num_theta_data = len(simulator.indices_to_consider) * num_theta_multiplier
gen_meth_theta = Gen_meth_enum(gen_meth_theta)
# Note at present, training data is always the same between jobs since we set the data generation seed to 1
sim_data = simulator.gen_sim_data(
    num_theta_data,
    num_x_data,
    gen_meth_theta,
    gen_meth_x,
    sep_fact,
    set_seed,
    False,
    x_vals
)

# print(sim_data.y_vals)
# print(sim_data.get_unique_theta())
# # Gen sse_sim_data and sse_sim_val_data
sim_sse_data = simulator.sim_data_to_sse_sim_data(
    method, sim_data, exp_data, sep_fact, False
)

# Generate validation data if applicable. This is only useful for small (<4 Params + 1 State Point). Otherwise this takes up too much memory
if num_val_pts > 0:
    gen_meth_theta_val = Gen_meth_enum(
        gen_meth_theta_val
    )  # input is an integer (1 or 2)
    val_data = simulator.gen_sim_data(
        num_val_pts,
        num_x_data,
        gen_meth_theta_val,
        gen_meth_x,
        sep_fact,
        set_seed,
        True,
        x_vals
    )
    val_sse_data = simulator.sim_data_to_sse_sim_data(
        method, val_data, exp_data, sep_fact, True
    )
# Set validation data to None if not generating it
else:
    val_data = None
    val_sse_data = None
    gen_meth_theta_val = gen_meth_theta_val  # Value is None

# Define cs_name and cs_params class
cs_name = get_cs_class_from_val(cs_name_val).name #Save name of case study here
# Signac saves all BO_Results in different folders, so they can have the same name
cs_params = CaseStudyParameters(
    cs_name,
    ep0,
    sep_fact,
    normalize,
    kernel,
    lenscl,
    outputscl,
    retrain_GP,
    reoptimize_obj,
    gen_heat_map_data,
    bo_iter_tot,
    bo_runs_in_job,
    save_data,
    None,
    seed,
    ei_tol,
    obj_tol,
)
# Initialize driver class
driver = GPBO_Driver(
    cs_params,
    method,
    simulator,
    exp_data,
    sim_data,
    sim_sse_data,
    val_data,
    val_sse_data,
    None,
    ep_bias,
    gen_meth_theta,
)
# Get results
gpbo_res_simple, gpbo_res_GP = driver.run_bo_restarts()

[ 9.61356984e-03 -1.35428561e-02 -2.11561516e-03 -2.51235296e-03
  7.48420613e-03 -7.12119852e-03  6.91286035e-04  1.85894830e-01
  1.00027743e+00  1.00383184e+00 -7.23624971e-03  7.52622413e-03
  5.92770826e-03  2.33186176e-01  9.81889438e-01  9.94075176e-01
  9.98990231e-01  9.93784509e-01  9.98222621e-01  1.00347775e+00
  1.55422556e-02  5.60560639e-02  1.09663157e-01  2.53778634e-01
  4.02456005e-01  6.14589396e-01  8.49124818e-01  9.59007053e-01
  1.00176131e+00  9.98697254e-01  9.92056052e-01  9.78494312e-01
  9.65722672e-01  9.32780952e-01  8.51417522e-01  7.30688591e-01
  5.73938491e-01  4.61829865e-01  3.11504967e-01  2.67272231e-01
  1.97353402e-03 -2.31544630e-03 -7.41137694e-03  7.54707058e-02
  5.73580882e-01  8.95015198e-01  9.75599894e-01  9.94099918e-01
  9.98321645e-01  1.00373181e+00 -4.95994618e-03  8.23724228e-03
  1.16992912e-02  9.98028958e-01  1.00321170e+00  9.99503121e-01
  1.00744013e+00  1.00999235e+00  1.01436952e+00  9.90818444e-01]
[[ 1627.261251   -4876.1

In [ ]:
#Define method, ep_enum classes, indecies to consider, and kernel
meth_name = Method_name_enum(meth_name_val)
method = GPBO_Methods(meth_name)
ep_enum = Ep_enum(ep_enum_val)
kernel = Kernel_enum(kernel_enum_val)

#Define Simulator Class (Export your Simulator Object Here)
simulator = simulator_helper_test_fxns(cs_name_val, noise_mean, noise_std, seed)

#Generate Exp Data (OR Add your own experimental data as a Data class object)
set_seed = 1 #Set set_seed to 1 for data generation
gen_meth_x = Gen_meth_enum(gen_meth_x)
# x_vals = None
x_vals =np.array([0.0087,0.0269,0.0568,0.1556,0.2749,0.4449,0.661,0.8096,0.9309,0.9578]) #CS17
# x_vals = np.array([0.0,0.1115,0.2475,0.4076,0.5939,0.8230,0.9214,0.9296,0.985,1.000]) #CS16
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x, set_seed, x_vals, 0.01)

#Set simulator noise_std artifically as 5% of y_exp mean (So that noise will be set rather than trained)
simulator.noise_std = np.abs(np.mean(exp_data.y_vals))*0.01

#Create Exploration Bias Class
if ep_enum.value == 1:
    #Constant value stays constant
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, None, None, None, None, None, None)
elif ep_enum.value == 2:
    #For decay method, decay from mixed to full exploitation (alpha of 0.5) for this example
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, bo_iter_tot, None, 0.5, None, None, None)
elif ep_enum.value == 3:
    #Set ep multiplier to 1.5 as recommended in Boyle
    ep_bias = Exploration_Bias(ep0, None, ep_enum, None, None, 1.5, None, None, None, None)
elif ep_enum.value == 4:
    #Jasrasaria method will take care of itself
    ep_bias = Exploration_Bias(None, None, ep_enum, None, None, None, None, None, None, None)
else:
    raise Warning("Ep_enum value must be between 1 and 4!")
    
#Generate Sim (Training) Data (OR Add your own training data here as a Data class object)
num_theta_data = len(simulator.indices_to_consider)*num_theta_multiplier
gen_meth_theta = Gen_meth_enum(gen_meth_theta)
#Note at present, training data is always the same between jobs since we set the data generation seed to 1
sim_data = simulator.gen_sim_data(num_theta_data, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, set_seed, False, x_vals)

# #Gen sse_sim_data and sse_sim_val_data
sim_sse_data = simulator.sim_data_to_sse_sim_data(method, sim_data, exp_data, sep_fact, False)

#Generate validation data if applicable. This is only useful for small (<5 Params + State Points). Otherwise this takes up too much memory
if num_val_pts > 0:
    gen_meth_theta_val = Gen_meth_enum(gen_meth_theta_val) #input is an integer (1 or 2)
    val_data = simulator.gen_sim_data(num_val_pts, num_x_data, gen_meth_theta_val, gen_meth_x, sep_fact, set_seed, True)
    val_sse_data = simulator.sim_data_to_sse_sim_data(method, val_data, exp_data, sep_fact, True)        
#Set validation data to None if not generating it
else:
    val_data = None
    val_sse_data = None
    gen_meth_theta_val = gen_meth_theta_val #Value is None
                    
#Define cs_name and cs_params class
#Signac saves all BO_Results in different folders, so they can have the same name
cs_name = "BO_Results"
cs_params = CaseStudyParameters(cs_name, ep0, sep_fact, normalize, kernel, lenscl, outputscl,
                                retrain_GP, reoptimize_obj, gen_heat_map_data, bo_iter_tot,
                                bo_runs_in_job, True, None, seed, ei_tol, obj_tol)
#Initialize driver class
driver = GPBO_Driver(cs_params, method, simulator, exp_data, sim_data, sim_sse_data, val_data, val_sse_data, None, ep_bias, gen_meth_theta)
#Get results
restart_bo_results, gp_stuff = driver.run_bo_restarts()

In [ ]:
# restart_bo_results[0].results_df.iloc[:,0:13]
#['Theta Min Obj', 'Theta Obj Act Cum', 'Min Obj Act', "Min Obj Act Cum"]